# Walking friction surface creation - 100m resample version

Clean up various rasters and combining them into a walking friction surface at 100m resolution. This will later be merged with an on-road friction surface for a final product.

In [1]:
import os, sys

from datetime import datetime

import numpy as np
from numpy import pi, log, tan, empty, float32, arctan, rad2deg, gradient
from numpy import arctan2, reshape, where
from scipy.ndimage import gaussian_gradient_magnitude

import rasterio
from rasterio import features, transform
from rasterio.mask import mask
from rasterio.transform import Affine
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.io import MemoryFile

import pandas as pd
import geopandas as gpd

import shapely
from shapely.geometry import shape, box, Polygon

In [2]:
data_dir = r'P:\PAK\\Code\Accessibility\Inputs'
in_dem = r'P:\PAK\\GEO\Elevation'
in_lc = r'P:\PAK\\GEO\Landcover\ICIMOD'
in_river = r'P:\PAK\\Code\Accessibility\OSM_River_Geofabrik'
in_roads = r'P:\PAK\\Code\Accessibility\Osm_Roads_Geofabrik'
out_dir = r'P:\PAK\\Code\Accessibility\Intermediate_Rasters'
fric_dir = r'P:\PAK\\Code\Accessibility\Friction_Surface'

Projections

In [3]:
dest_crs = 'EPSG:32642' # this is a Pakistani UTM projection, assign correct projection for project area

Today's date

In [4]:
today = datetime.today().strftime('%y%m%d')

Load Shapefile of KP to clip the final data

In [6]:
kp = gpd.read_file(r'P:\PAK\GEO\Boundaries\OCHA\pak_admbnda_adm1_ocha_pco_gaul_20181218.shp')

In [7]:
kp = kp[kp['ADM1_EN'] == 'Khyber Pakhtunkhwa']
kp = kp.to_crs(dest_crs)

In [8]:
# Buffer the polygon by 20km so we take in nearby markets and roads that may be used
kp.geometry = kp.buffer(20000)

In [9]:
kp.head()

,Shape_Leng,Shape_Area,ADM1_EN,ADM1_PCODE,ADM1_REF,ADM1ALT1EN,ADM1ALT2EN,ADM0_EN,ADM0_PCODE,date,validOn,validTo,geometry
2,27.479368,9.901186,Khyber Pakhtunkhwa,PK2,None,None,None,Pakistan,PK,2018-11-30,2018-12-18,None,"POLYGON ((505910.537 3561734.747, 505840.700 3..."


Useful functions

In [10]:
# Lightly adapted from https://gis.stackexchange.com/questions/290030/what-does-it-mean-to-reproject-a-satellite-image-from-utm-zone-13n-to-wgs84

def reproject_tif(source_file, destination_file, dest_crs):
    """Re-projects tif at source file to destination CRS at destination file.

    Args:
        source_file: file to re-project
        destination_file: file to store re-projection

    Returns:
        destination_file: where the re-projected file is saved at
    """

    with rasterio.open(source_file) as src:
        dst_crs = dest_crs
        transform, width, height = calculate_default_transform(
            src.crs,
            dst_crs,
            src.width,
            src.height,
            *src.bounds
        )

        kwargs = src.meta.copy()
        kwargs.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height,
            "compress":'LZW'
        })

        with rasterio.open(destination_file, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest,
                    num_threads=-1
                )

        return destination_file

In [11]:
# slope calculation code from here: https://github.com/dgketchum/dem/blob/master/dem.py

def get_slope(dem, mode='percent'):
    slope = gaussian_gradient_magnitude(dem, 5, mode ='nearest')
    if mode == 'percent':
        pass
    if mode == 'fraction':
        slope = slope / 100
    if mode == 'degrees':
        slope = rad2deg(arctan(slope / 100))
    
    return slope
    

# Walking friction surface

We start by creating a walking friction surface. This forms half of the multi-modal friction surface and can also be used as a standalone analysis tool (for walking-only analysis).

## Reclassify landcover

In [12]:
# Build a "lookup array" where the index is the original value and the value
# is the reclassified value.  Setting all of the reclassified values is cheap 
# because the memory is only allocated once for the lookup array.

lookup = np.arange(256, dtype=np.float32)

# Replacement values are the divisors of walking speeds specific to that landcover type -- so, a value of 2 means "divide the walking speed by 2"
# THESE ARE EXAMPLE VALUES AND MUST BE REPLACED -- refer to Herzog article (Herzog 2020)

lookup[1] = 2
lookup[2] = 2
lookup[3] = 2
lookup[4] = 2
lookup[5] = 2
lookup[6] = 2
lookup[7] = 1.2
lookup[8] = 1.5
lookup[9] = 1.8
lookup[10] = 1.5
lookup[11] = 1.2
lookup[12] = 1.19
lookup[13] = 1.6
lookup[14] = 5
lookup[255] = 1.5


In [13]:
with rasterio.open(os.path.join(in_lc,'pakistan_icimod_landcover_2010_32642.tif')) as lc_src:
    
    # define scale factors
    scale_factor_x = np.abs((100 / lc_src.transform.a))
    scale_factor_y = np.abs((100 / lc_src.transform.e))
    
    # Read and transform to 100m x 100m resolution
    lc_array = lc_src.read(1,
                           out_shape=(
                               lc_src.count,
                               int(lc_src.height / scale_factor_y) ,
                               int(lc_src.width / scale_factor_x) ),
                           resampling=Resampling.mode
                          )
                           
    lc_transform = lc_src.transform * lc_src.transform.scale(
            (scale_factor_x),
            (scale_factor_y)
        )
    
    # Reclassify in a single operation using broadcasting
    lc_array = np.where(lc_array < 15, lookup[lc_array], 255).astype(np.float32)

    # Update the profile to reflect transformation
    lc_profile = lc_src.profile
    
    lc_profile.update({
    "dtype": 'float32',
    "height" : lc_array.shape[0],
    "width" : lc_array.shape[1],
    "transform" : lc_transform
})

In [14]:
lc_profile

{'driver': 'GTiff', 'dtype': 'float32', 'nodata': 255.0, 'width': 8277, 'height': 6855, 'count': 1, 'crs': CRS.from_epsg(32642), 'transform': Affine(100.0, 0.0, 489325.4971,
       0.0, -100.0, 4128951.9286), 'tiled': False, 'compress': 'lzw', 'interleave': 'band'}

In [15]:
lc_transform

Affine(100.0, 0.0, 489325.4971,
       0.0, -100.0, 4128951.9286)

In [85]:
res = str(int(np.abs(lc_transform.e))) + 'm'

In [86]:
res

'100m'

## Rivers and bridges as obstacles

Download rivers, transform to geodataframe in correct projection

In [16]:
# local file import
rivs = gpd.read_file(os.path.join(in_river,"osm_river_utm.shp"),driver="ESRI Shapefile")

# minor cleanup
rivs = rivs.reset_index()
rivs_slim = rivs[['geometry']]
rivs_slim['exist'] = 0
rivs_slim = rivs_slim.to_crs(dest_crs)

C:\Users\wb467985\Anaconda3\envs\geo\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [18]:
# create a generator containing geometry, value pairs for rivers

riv_shapes = ((geom,exist) for geom, exist in zip(rivs_slim.geometry,rivs_slim['exist']))

# This will give the raster the size and dimensions of the landcover raster -- areas not covered by rivers will be 1.

riv_rast = features.rasterize(riv_shapes,\
                  out_shape = (lc_profile['height'],\
                               lc_profile['width']),\
                  transform=lc_profile['transform'],
                  all_touched=True,
                  fill=1,
                  dtype = np.float32)


**Optional step, change the `using_bridges` parameter to trigger / disable**:
</br>Download bridges, transform to geodataframe in correct projection, snap to nearest river

In [19]:
# change this to trigger / disable sequence below 
using_bridges = 'no'

In [20]:
if using_bridges == 'yes':

    brdgs = gpd.read_file(os.path.join(data_dir,vect_dir,"bridges_osm_210710.shp"),driver="ESRI Shapefile")
    brdgs = brdgs.to_crs(dest_crs)

    # snapping to rivers
    # from: https://gis.stackexchange.com/questions/306838/snap-points-shapefile-to-line-shapefile-using-shapely</br>

    brdgs_snapped = [shapely.ops.snap(i,j, 2000) for i, j in zip(brdgs['geometry'],rivs_slim['geometry']) ]

    # alternately
    # from shapely.ops import nearest_points
    # rivs_slim_unary = rivs_slim.geometry.unary_union
    # brdgs_snapped = brdgs.geometry.apply(lambda x: rivs_slim_unary.interpolate(rivs_slim_unary.project(x)))

    brdgs_snapped = gpd.GeoDataFrame(brdgs_snapped)
    brdgs_snapped.rename({0:'geometry'},axis=1,inplace=True)
    brdgs_snapped = brdgs_snapped.set_geometry('geometry').set_crs(dest_crs)
    brdgs_snapped['exist'] = 1

    # brdgs_snapped.to_file(os.path.join(data_dir,vect_dir,"bridges_osm_210710.shp"),driver="ESRI Shapefile")

    # generator of vector shapes and values (existean)

    brdg_shapes = ((geom,exist) for geom, exist in zip(brdgs_snapped.geometry,brdgs_snapped['exist']))

    # Create a bridge raster layer we can overlay on the rivers -- areas not covered by bridges will be 0

    brdg_rast = features.rasterize(brdg_shapes,\
                      out_shape = (lc_profile['height'],\
                                   lc_profile['width']),\
                      transform=lc_profile['transform'],
                      all_touched=True,
                      fill=0,
                      dtype = np.float32)

    # Add rasters together and export as final river raster layer
    
    riv_rast_fin = riv_rast + brdg_rast # add rasters together so bridges punch "holes" in rivers

else:
    # If not using bridges
    riv_rast_fin = riv_rast

In [21]:
riv_rast_fin

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]], dtype=float32)

In [23]:
riv_rast_fin.dtype

dtype('float32')

## Roads to walking surface mask raster

We assume that people walking on roads and paths are not affected by landcover. To model this we turn roads into a raster with value = 1 (for 1 * speed). Then we merge it with the landcover raster for a final walking speed modifier raster

In [24]:
rds = gpd.read_file(os.path.join(data_dir,'master_transport_Sep23.gpkg'),driver="GPKG")

In [25]:
# assign 1 value to represent existence of road
rds['exist'] = 1

# generator of vector shapes and values (boolean)
rds_shapes = ((geom,exist_val) for geom, exist_val in zip(rds.geometry,rds['exist']))

# This will give the raster the size and dimensions of the landcover raster -- areas not covered by roads will be 0.

rd_mask_rast = features.rasterize(rds_shapes,\
                  out_shape = (lc_profile['height'],\
                               lc_profile['width']),\
                  transform=lc_profile['transform'],
                  all_touched=True,
                  fill=0,
                  dtype = np.float32)


In [26]:
rd_mask_rast.shape

(6855, 8277)

First combine the rivers with the landcover raster, inserting a `600000` divider where rivers exist, so crossing rivers without a bridge has a huge cost. Then combine with the road mask, inserting a `1` multiplier where roads are. The order is important, so roads overwrite rivers (implicitly via bridges, which are not reliably recorded in many roads datasets)
</br></br>Note that if landcover *multipliers* instead of *dividers* are used, you need to invert this and use a very small decimal value for the rivers.

In [27]:
walkspeed_mod_rast = np.where(riv_rast_fin == 0, 600000, lc_array)

In [28]:
walkspeed_mod_rast = np.where(rd_mask_rast == 1, 1, walkspeed_mod_rast)

In [29]:
walkspeed_mod_rast.shape

(6855, 8277)

In [30]:
walkspeed_mod_rast.dtype

dtype('float64')

In [31]:
np.min(walkspeed_mod_rast)

1.0

## Base walking speeds from DEM

#### DEM to slope

First import the DEM and transform it to the same CRS, cell resolution, and dimensions as the landcover layer. This enables raster math between the layers and any other arrays derived from them.

In [32]:
with rasterio.open(os.path.join(in_dem,'DEM_KPK_UTM.tif')) as dem_src:
    # Read as numpy array
    dem_array = dem_src.read(1)
    dem_transform = dem_src.transform
    dem_profile = dem_src.profile


In [33]:
# must reproject to the same projection and cell-size as the landcover raster

# create a blank array of the correct dimensions to populate while reprojecting
dem_array_reproj = np.zeros(lc_array.shape, np.float32)

# reproject uing a cubic reprojection for smoothness

with rasterio.Env():
    reproject(
        dem_array,
        dem_array_reproj,
        src_transform=dem_transform,
        src_crs=dem_profile['crs'],
        dst_transform=lc_profile['transform'],
        dst_crs=lc_profile['crs'],
        resampling=Resampling.cubic)

In [34]:
dem_array_reproj.shape


(6855, 8277)

In [35]:
slope = get_slope(dem_array_reproj,mode='fraction')

In [36]:
# remove artefacts that will produce slopes > 100%
slope = np.where(slope>1,1,slope)

In [37]:
slope.dtype

dtype('float32')

In [38]:
slope

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.02620982, 0.02875681, 0.03057061, ..., 0.        , 0.        ,
        0.        ],
       [0.02473836, 0.02724167, 0.02902424, ..., 0.        , 0.        ,
        0.        ],
       [0.0242059 , 0.02645768, 0.0280408 , ..., 0.        , 0.        ,
        0.        ]], dtype=float32)

Calculate walking speeds over the slope using Irmischer-Clarke's walking speed formula.

In [39]:
# Irmischer-Clarke have a generic off-road speed but we don't use this given that we adjust by specific landcover type.  We stick to their on-road speed and modify that.
# We include the I-C off-road below for reference

# walkspeed_offroad = (0.11 + (0.67 * np.exp(-np.square((slope*100) + 2) / 3600))) * 3.6 # I-C off-road
walkspeed_onroad = (0.11 + np.exp(-np.square((slope*100) + 5) / 3600)) * 3.6

In [40]:
walkspeed_onroad

array([[3.9710863, 3.9710863, 3.9710863, ..., 3.9710863, 3.9710863,
        3.9710863],
       [3.9710863, 3.9710863, 3.9710863, ..., 3.9710863, 3.9710863,
        3.9710863],
       [3.9710863, 3.9710863, 3.9710863, ..., 3.9710863, 3.9710863,
        3.9710863],
       ...,
       [3.9383864, 3.9345047, 3.9316657, ..., 3.9710863, 3.9710863,
        3.9710863],
       [3.9405727, 3.9368286, 3.93409  , ..., 3.9710863, 3.9710863,
        3.9710863],
       [3.9413538, 3.938014 , 3.9356081, ..., 3.9710863, 3.9710863,
        3.9710863]], dtype=float32)

In [41]:
walkspeed_base = walkspeed_onroad
# walkspeed_base = np.where(rd_mask_rast == 1,walkspeed_onroad,walkspeed_offroad) # included for reference purposes, in situations where you don't want to adjust by landcover

In [42]:
np.min(walkspeed_base)

0.5643742

In [43]:
np.max(walkspeed_base)

3.9710863

In [44]:
walkspeed_base.shape

(6855, 8277)

#### Vertical distances

Calculate the additional vertical distance covered when crossing a cell (the rise, in addition to the run represented by the cell's resolution).

In [45]:
slope

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.02620982, 0.02875681, 0.03057061, ..., 0.        , 0.        ,
        0.        ],
       [0.02473836, 0.02724167, 0.02902424, ..., 0.        , 0.        ,
        0.        ],
       [0.0242059 , 0.02645768, 0.0280408 , ..., 0.        , 0.        ,
        0.        ]], dtype=float32)

In [46]:
vert_dist_simple = 1 / np.cos(slope)
# vert_dist_simple = np.where(((vert_dist_simple<0) | (vert_dist_simple>2.5)),2.5,vert_dist_simple)

In [47]:
vert_dist_simple.shape

(6855, 8277)

Calculate the additional distance associated with zig-zagging paths - the zig goes sideways halfway up the cell, the zag sideways up the other half. We do not consider zig-zags that go more than 2 ways up a cell

In [48]:
# the switchback cutoff value is somewhat arbitrary and perhaps even culturally defined. We use one of the higher values found in the literature

switchback_cutoff = 0.30

In [49]:
vert_dist_switchback = np.tan(slope) / np.sin(switchback_cutoff)

Combine the two arrays into one walking cost array, forcing walkers to use zig-zagging switchbacks while crossing terrain above a cutoff slope of `30%` (0.30). 

In [50]:
vert_dist_final = np.where(slope <= switchback_cutoff,vert_dist_simple,vert_dist_switchback)

In [51]:
vert_dist_final

array([[1.       , 1.       , 1.       , ..., 1.       , 1.       ,
        1.       ],
       [1.       , 1.       , 1.       , ..., 1.       , 1.       ,
        1.       ],
       [1.       , 1.       , 1.       , ..., 1.       , 1.       ,
        1.       ],
       ...,
       [1.0003436, 1.0004137, 1.0004674, ..., 1.       , 1.       ,
        1.       ],
       [1.000306 , 1.0003712, 1.0004213, ..., 1.       , 1.       ,
        1.       ],
       [1.000293 , 1.0003501, 1.0003933, ..., 1.       , 1.       ,
        1.       ]], dtype=float32)

In [52]:
vert_dist_final[vert_dist_final < 1]

array([], dtype=float32)

In [53]:
# vert_dist_final = np.where(((vert_dist_final<0) | (vert_dist_final>2.5)),2.5,vert_dist_final)

In [54]:
vert_dist_final.shape

(6855, 8277)

In [55]:
# # optional -- write out the vertical-horizontal for future use

lc_profile.update({"dtype":'float64',\
                   "COMPRESS":'ZSTD',
                   "nodata":-99999})

with rasterio.open(os.path.join(out_dir,f'KP_Combined_VertHorizontal_Distance_Simple_{today}_100m.tif'),'w',**lc_profile) as dst1:
     dst1.write(vert_dist_simple,indexes=1)
        
with rasterio.open(os.path.join(out_dir,f'KP_Combined_VertHorizontal_Distance_w_Switchbacks_{today}_100m.tif'),'w',**lc_profile) as dst2:
     dst2.write(vert_dist_final,indexes=1)

## Merge rasters into final walking friction surface

Combine the various arrays into a final walking friction surface in 6 stages:
1. Multiply the base walking speed computed from the DEM Slope by the speed modifier
2. Create a monsoon walking speed as 0.75 of the base walking speed and the winter walking speed similarly, using a multiplier determined by elevation
3. Adjust the speeds for altitude
4. Transform these speeds into friction values
5. Multiply the friction values by the vert/horizontal multiplication factor (e.g. 1.5)
6. Convert extraneous values to -99999 nodata values

In [61]:
walkspeed_base

array([[3.9710863, 3.9710863, 3.9710863, ..., 3.9710863, 3.9710863,
        3.9710863],
       [3.9710863, 3.9710863, 3.9710863, ..., 3.9710863, 3.9710863,
        3.9710863],
       [3.9710863, 3.9710863, 3.9710863, ..., 3.9710863, 3.9710863,
        3.9710863],
       ...,
       [3.9383864, 3.9345047, 3.9316657, ..., 3.9710863, 3.9710863,
        3.9710863],
       [3.9405727, 3.9368286, 3.93409  , ..., 3.9710863, 3.9710863,
        3.9710863],
       [3.9413538, 3.938014 , 3.9356081, ..., 3.9710863, 3.9710863,
        3.9710863]], dtype=float32)

In [62]:
walkspeed_dry_step1 = np.divide(walkspeed_base,walkspeed_mod_rast).astype(np.float32)

# Monsoon mod
walkspeed_msn_step1 = np.multiply(walkspeed_dry_step1,0.75).astype(np.float32)

#Winter Speed Cutoff
walkspeed_winter_step0 = np.where(dem_array_reproj <= 2350, walkspeed_dry_step1, (np.multiply(walkspeed_dry_step1,0.75)))
walkspeed_winter_step1 = np.where(dem_array_reproj <= 3000, walkspeed_winter_step0, (np.multiply(walkspeed_dry_step1,0.6))).astype(np.float32)

In [63]:
np.max(walkspeed_dry_step1)

3.9710858

In [64]:
walkspeed_dry_step1

array([[0.01557289, 0.01557289, 0.01557289, ..., 0.01557289, 0.01557289,
        0.01557289],
       [0.01557289, 0.01557289, 0.01557289, ..., 0.01557289, 0.01557289,
        0.01557289],
       [0.01557289, 0.01557289, 0.01557289, ..., 0.01557289, 0.01557289,
        0.01557289],
       ...,
       [0.01544465, 0.01542943, 0.0154183 , ..., 0.01557289, 0.01557289,
        0.01557289],
       [0.01545323, 0.01543854, 0.0154278 , ..., 0.01557289, 0.01557289,
        0.01557289],
       [0.01545629, 0.01544319, 0.01543376, ..., 0.01557289, 0.01557289,
        0.01557289]], dtype=float32)

Adjust walkspeeds by altitude

In [65]:
# We adjust altitude in two steps based on a literature review into how lower oxygen content at altitude affects walking speeds. Note this is not the best documented subject, at least in terms we can computer into a friction surface.
# This formula could probably be streamlined so that this step is condensed into one move
# The Global Friction Surface has just one formula but I found its high altitude (>5000) modifiers to be a little low compared to the available literature on athletic performance at altitude. Not a big deal except if you're working in the Himalayas

alt_adjust_dry_under3k = np.where(dem_array_reproj <= 2350, walkspeed_dry_step1, ((walkspeed_dry_step1) / (1 + ((dem_array_reproj - 2350)/5000))) )
walkspeed_dry_step2 = np.where(dem_array_reproj <= 3000, alt_adjust_dry_under3k, ((walkspeed_dry_step1) / (0.323 * np.exp((.00042*dem_array_reproj)))) )

alt_adjust_msn_under3k = np.where(dem_array_reproj <= 2350, walkspeed_msn_step1, ((walkspeed_msn_step1) / (1 + ((dem_array_reproj - 2350)/5000))) )
walkspeed_msn_step2 = np.where(dem_array_reproj <= 3000, alt_adjust_msn_under3k, ((walkspeed_msn_step1) / (0.323 * np.exp((.00042*dem_array_reproj)))) )

alt_adjust_winter_under3k = np.where(dem_array_reproj <= 2350, walkspeed_winter_step1, ((walkspeed_winter_step1) / (1 + ((dem_array_reproj - 2350)/5000))) )
walkspeed_winter_step2 = np.where(dem_array_reproj <= 3000, alt_adjust_winter_under3k, ((walkspeed_winter_step1) / (0.323 * np.exp((.00042*dem_array_reproj)))) )

In [66]:
np.min(walkspeed_dry_step2)

5.989555e-07

In [67]:
# refactor walking speeds to friction values in units of cell size / hour (e.g. 30m / hour)

friction_walk_dry_step1 = (1 / walkspeed_dry_step2) / (1000 / lc_transform.a)
friction_walk_msn_step1 = (1 / walkspeed_msn_step2) / (1000 / lc_transform.a)
friction_walk_winter_step1 = (1 / walkspeed_winter_step2) / (1000 / lc_transform.a)

In [68]:
# now multiply the friction surface by the merged vertical/horizontal distance to calculate the final friction surface
friction_walk_dry_final = np.multiply(friction_walk_dry_step1,vert_dist_final)
friction_walk_msn_final = np.multiply(friction_walk_msn_step1,vert_dist_final)
friction_walk_winter_final = np.multiply(friction_walk_winter_step1,vert_dist_final)

In [69]:
friction_walk_dry_final

array([[6.421417 , 6.421417 , 6.421417 , ..., 6.421417 , 6.421417 ,
        6.421417 ],
       [6.421417 , 6.421417 , 6.421417 , ..., 6.421417 , 6.421417 ,
        6.421417 ],
       [6.421417 , 6.421417 , 6.421417 , ..., 6.421417 , 6.421417 ,
        6.421417 ],
       ...,
       [6.4769573, 6.483801 , 6.4888325, ..., 6.421417 , 6.421417 ,
        6.421417 ],
       [6.4731207, 6.4796996, 6.4845347, ..., 6.421417 , 6.421417 ,
        6.421417 ],
       [6.4717546, 6.477613 , 6.481852 , ..., 6.421417 , 6.421417 ,
        6.421417 ]], dtype=float32)

In [251]:
# friction_walk_msn_final

Weed out Inf values and super high river values

In [70]:
# we use 1 as an arbitrary cutoff on the assumption that it will never actually take 1 hour to cross a grid cell, so values above that are bogus and filterable

friction_walk_dry_final = np.where(friction_walk_dry_final > 1, 10, friction_walk_dry_final)
friction_walk_msn_final = np.where(friction_walk_msn_final > 1, 10, friction_walk_msn_final)
friction_walk_winter_final = np.where(friction_walk_winter_final > 1, 10, friction_walk_winter_final)

Round up and change to float32 to reduce file sizes

In [71]:
friction_walk_dry_final = np.round(friction_walk_dry_final,8).astype(np.float32)
friction_walk_msn_final = np.round(friction_walk_msn_final,8).astype(np.float32)
friction_walk_winter_final = np.round(friction_walk_winter_final,8).astype(np.float32)

In [72]:
friction_walk_dry_final

array([[10., 10., 10., ..., 10., 10., 10.],
       [10., 10., 10., ..., 10., 10., 10.],
       [10., 10., 10., ..., 10., 10., 10.],
       ...,
       [10., 10., 10., ..., 10., 10., 10.],
       [10., 10., 10., ..., 10., 10., 10.],
       [10., 10., 10., ..., 10., 10., 10.]], dtype=float32)

Write out the final walking friction surface array as a raster

In [73]:
export_profile = lc_profile
export_profile.update({"dtype":'float32',\
                       "COMPRESS":'ZSTD',
                       "NUM_THREADS":'ALL_CPUS',
                       "nodata":-99999})

In [338]:
# with rasterio.open(os.path.join(fric_dir,f'KP_friction_walk_dry_{today}_{res}.tif'),'w',**export_profile) as dst:
#     dst.write(friction_walk_dry_final,indexes=1)
#     dst.build_overviews = ([2,4,8,10,14,16],Resampling.nearest) # build pyramids for quick viewing in desktop GIS software
    
# with rasterio.open(os.path.join(out_dir,f'KP_friction_walk_msn_{today}_{res}.tif'),'w',**export_profile) as dst:
#     dst.write(friction_walk_msn_final,indexes=1)
#     dst.build_overviews = ([2,4,8,10,14,16],Resampling.nearest) # build pyramids for quick viewing in desktop GIS software
        
# with rasterio.open(os.path.join(fric_dir,f'KP_friction_walk_winter_{today}_{res}.tif'),'w',**export_profile) as dst:
#     dst.write(friction_walk_winter_final)
#     dst.build_overviews = ([2,4,8,10,14,16],Resampling.nearest) # build pyramids for quick viewing in desktop GIS software   

**Cropped version of Friction_Walk**

In [81]:
# Clip the friction array by the buffered KP outline
# use a MemoryFile to avoid lots of IO -- otherwise have to save down, then load back up.

with MemoryFile() as memfile_dry:
    with memfile_dry.open(**export_profile) as fric_walk_dry_in_mem:
        fric_walk_dry_in_mem.write(friction_walk_dry_final,indexes=1)

        # Crop the memfile with the new shape `coords2`
        friction_walk_dry_final_mask, friction_walk_dry_mask_tform = mask(fric_walk_dry_in_mem, kp.geometry, crop=True)

# Monsoon
with MemoryFile() as memfile_msn:
    with memfile_msn.open(**export_profile) as fric_walk_msn_in_mem:
        fric_walk_msn_in_mem.write(friction_walk_msn_final,indexes=1)

        # Crop the memfile with the new shape `coords2`
        friction_walk_msn_final_mask, friction_walk_msn_mask_tform = mask(fric_walk_msn_in_mem, kp.geometry, crop=True)
        
# Winter

with MemoryFile() as memfile_winter:
    with memfile_winter.open(**export_profile) as fric_walk_winter_in_mem:
        fric_walk_winter_in_mem.write(friction_walk_winter_final,indexes=1)

        # Crop the memfile with the new shape `coords2`
        friction_walk_winter_final_mask, friction_walk_winter_mask_tform = mask(fric_walk_winter_in_mem, kp.geometry, crop=True)

In [82]:
# ensure it's float32 so the output size is smaller
friction_walk_dry_final_mask = friction_walk_dry_final_mask.astype(np.float32)
friction_walk_msn_final_mask = friction_walk_msn_final_mask.astype(np.float32)
friction_walk_winter_final_mask = friction_walk_winter_final_mask.astype(np.float32)

In [87]:
# make a new profile for export, inserting the new height/width/transform values from the clipped raster

mask_profile = export_profile.copy()

mask_profile.update({"dtype" : 'float32',
                     "height":friction_walk_winter_final_mask.shape[1],
                     "width":friction_walk_winter_final_mask.shape[2],
                     "transform" : friction_walk_winter_mask_tform})

In [88]:
# Export the three seasons

with rasterio.open(os.path.join(fric_dir,f'KP_friction_walk_dry_{today}_{res}_masked.tif'),'w',**mask_profile) as dst:
    dst.write(friction_walk_dry_final_mask)
    dst.build_overviews = ([2,4,8,10,14,16],Resampling.nearest) # build pyramids for quick viewing in desktop GIS software
    
with rasterio.open(os.path.join(fric_dir,f'KP_friction_walk_msn_{today}_{res}_masked.tif'),'w',**mask_profile) as dst:
    dst.write(friction_walk_msn_final_mask)
    dst.build_overviews = ([2,4,8,10,14,16],Resampling.nearest) # build pyramids for quick viewing in desktop GIS software

with rasterio.open(os.path.join(fric_dir,f'KP_friction_walk_winter_{today}_{res}_masked.tif'),'w',**mask_profile) as dst:
    dst.write(friction_walk_winter_final_mask)
    dst.build_overviews = ([2,4,8,10,14,16],Resampling.nearest) # build pyramids for quick viewing in desktop GIS software

Mask, then export VH distance layers as well

In [92]:
# mask first

with MemoryFile() as memfile_dry:
    with memfile_dry.open(**export_profile) as vert_dist_simple_in_mem:
        vert_dist_simple_in_mem.write(vert_dist_simple,indexes=1)

        # Crop the memfile with the new shape
        vert_dist_simple_mask, vert_dist_simple_mask_tform = mask(vert_dist_simple_in_mem, kp.geometry, crop=True)

        
# Crop to KP extent
with MemoryFile() as memfile_dry:
    with memfile_dry.open(**export_profile) as vert_dist_final_in_mem:
        vert_dist_final_in_mem.write(vert_dist_final,indexes=1)

        # Crop the memfile with the new shape
        vert_dist_final_mask, vert_dist_final_mask_tform = mask(vert_dist_final_in_mem, kp.geometry, crop=True)


In [94]:
# # write out the vertical-horizontal for use in Step 4

with rasterio.open(os.path.join(out_dir,f'KP_Combined_VertHorizontal_Distance_Simple_{today}_{res}_masked.tif'),'w',**mask_profile) as dst1:
     dst1.write(vert_dist_simple_mask)
        
with rasterio.open(os.path.join(out_dir,f'KP_Combined_VertHorizontal_Distance_w_Switchbacks_{today}_{res}_masked.tif'),'w',**mask_profile) as dst2:
     dst2.write(vert_dist_final_mask)

# Intermediate file export

It's easy to mistype a parameter and mess up the whole result. Looking through the intermediate files sequentially is usually the fastest way to catch and correct such errors. Below are quick code snippets to export these intermediate files.

In [13]:
with rasterio.open(
        os.path.join(out_dir,f'KP_LC_Modifier.tif'), 'w',**lc_profile) as dst:
    dst.write(lc_array)

In [22]:
with rasterio.open(
        os.path.join(out_dir,f'KP_River_Raster.tif'), 'w',**lc_profile) as dst:
    dst.write(riv_rast, indexes=1)

In [51]:
slope = slope.astype('float64')
dem_profile.update({"dtype":'float64',\
                    "compress":'LZW'})

In [58]:
# # optional -- write out the slope for future use
lc_profile.update({"dtype":'float64',\
                   "COMPRESS":'LZW',
                   "nodata":-99999})

with rasterio.open(os.path.join(out_dir,'KP_Slope.tif'),'w',**lc_profile) as dst:
     dst.write(slope)

In [ ]:
lc_profile.update({"dtype":'float64',\
                   "COMPRESS":'LZW',
                   "nodata":-99999})

with rasterio.open(os.path.join(out_dir,'KP_Walkspeed_base.tif'),'w',**lc_profile) as dst:
     dst.write(walkspeed_base)

In [30]:
lc_profile.update({"dtype":'float64',\
                   "COMPRESS":'LZW',
                   "nodata":-99999})

with rasterio.open(os.path.join(out_dir,'KP_Walkspeed_Modifier.tif'),'w',**lc_profile) as dst:
     dst.write(walkspeed_mod_rast)

ValueError: the array's dtype 'uint32' does not match the file's dtype 'float64'

In [54]:
lc_profile.update({"dtype":'float64',\
                   "COMPRESS":'ZSTD',
                   "nodata":-99999})

with rasterio.open(os.path.join(out_dir,f'KP_alt_adjust_dry_under3k_{today}.tif'),'w',**lc_profile) as dst:
     dst.write(alt_adjust_dry_under3k)

In [55]:
lc_profile.update({"dtype":'float64',\
                   "COMPRESS":'ZSTD',
                   "nodata":-99999})

with rasterio.open(os.path.join(out_dir,f'KP_walkspeed_dry_step2_new_{today}.tif'),'w',**lc_profile) as dst:
     dst.write(walkspeed_dry_step2)

In [56]:
#Temporary adjustment over 3k
temp_over3k = (0.323 * np.exp((.00042*dem_array_reproj)))

In [58]:
lc_profile.update({"dtype":'float64',\
                   "COMPRESS":'ZSTD',
                   "nodata":-99999})

with rasterio.open(os.path.join(out_dir,f'KP_temp_over3k_{today}.tif'),'w',**lc_profile) as dst:
     dst.write(temp_over3k)

In [57]:
temp_walkspeed_over3k = (walkspeed_dry_step1)/(0.323 * np.exp((.00042*dem_array_reproj)))

In [59]:
lc_profile.update({"dtype":'float64',\
                   "COMPRESS":'ZSTD',
                   "nodata":-99999})

with rasterio.open(os.path.join(out_dir,f'KP_temp_walkspeed_over3k_{today}.tif'),'w',**lc_profile) as dst:
     dst.write(temp_walkspeed_over3k)